# Prompt Token Compression

In [1]:
import datetime
import json
import time
import os
import datetime
import json
import time
from openai import AzureOpenAI
from dotenv import load_dotenv
import json
import copy
import textwrap

# Load environment variables
load_dotenv()

def aoai_call(prompt,model):
    client = AzureOpenAI(
        api_version=os.getenv("API_VERSION"),
        azure_endpoint=os.getenv("AZURE_ENDPOINT"),
        api_key=os.getenv("API_KEY")
    )

    start_time = time.time()

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )

    end_time = time.time()
    e2e_time = end_time - start_time

    result=json.loads(completion.model_dump_json(indent=2))
    prompt_tokens=result["usage"]["prompt_tokens"]
    completion_tokens=result["usage"]["completion_tokens"]
    completion_text=result["choices"][0]["message"]["content"]

    return result,prompt_tokens,completion_tokens,completion_text,e2e_time

model=os.getenv("MODEL35")

prompt_complex = open("./prompt_hardest.txt").read()

question = """Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?"""

answer="""The cost of the house and repairs came out to 80,000+50,000=$<<80000+50000=130000>>130,000
He increased the value of the house by 80,000*1.5=<<80000*1.5=120000>>120,000
So the new value of the house is 120,000+80,000=$<<120000+80000=200000>>200,000
So he made a profit of 200,000-130,000=$<<200000-130000=70000>>70,000
#### 70000"""




### Base Case: full, uncompressed prompt

In [2]:

instruction = "Please reference the following examples to answer the math question,\n"
prompt = instruction + prompt_complex + "\n\nQuestion: " + question


result,prompt_tokens,completion_tokens,completion_text,e2e_time=aoai_call(prompt,model)
print(f"Prompt Tokens: {prompt_tokens}")
print(f"Completion Tokens: {completion_tokens}")
print(f"Time taken: {e2e_time:.2f} seconds")
print(f"Completion text: {completion_text}")

Prompt Tokens: 2435
Completion Tokens: 110
Time taken: 3.85 seconds
Completion text: Let's think step by step
The value of the house increased by 150%, which means it increased by $80,000 * 150% = $120,000
The total value of the house after repairs is $80,000 + $120,000 = $200,000
Josh invested a total of $80,000 + $50,000 = $130,000 in the house
Therefore, his profit is $200,000 - $130,000 = $70,000
The answer is $70,000


### Applying Prompt Compression

In this example, the prompt compression has been completed on a virtual machine with a GPU. The compressed prompt has simply been stored in this notebook as text. To try out the entire process yourself, see the LLMLingua examples here:

https://github.com/microsoft/LLMLingua/tree/main/examples

In [3]:
# Commented out as this step requires a GPU, or if run on CPU can require complex dependencies

# # Setup LLMLingua
# from llmlingua import PromptCompressor

# llm_lingua = PromptCompressor()

In [4]:
# this is the output of the above step

compressed_prompt = {'compressed_prompt': 'Question: Sam bought a dozen boxes, each with 30 highlighter pens inside, for $10 each. He reanged five of boxes into packages of sixlters each and sold them $3 per. He sold the rest theters separately at the of three pens $2. How much did make in total, dollars?\nLets think step step\nSam bought 1 boxes x010 worth oflters.\nHe bought 12 * 00ters in\nSam then took 5 boxes 6ters0ters\nHe sold these boxes for 5 *5\nAfterelling these  boxes there were 300ters\nese form 30 /310 of three\n sold each for2 each, so made =0 from\n, he0 $15\nSince his original1 he earned $520 = $115 in profit.\nThe answer is 115',
 'origin_tokens': 2365,
 'compressed_tokens': 172,
 'ratio': '13.8x',
 'rate': '7.3%',
 'saving': ', Saving $0.1 in GPT-4.'}

In [5]:
instruction = "Please reference the following examples to answer the math question,\n"

prompt = instruction + compressed_prompt["compressed_prompt"] + "\n\nQuestion: " + question

result,prompt_tokens,completion_tokens,completion_text,e2e_time=aoai_call(prompt,model)
print(f"Prompt Tokens: {prompt_tokens}")
print(f"Completion Tokens: {completion_tokens}")
print(f"Time taken: {e2e_time:.2f} seconds")
print(f"Completion text: {completion_text}")

Prompt Tokens: 242
Completion Tokens: 165
Time taken: 3.83 seconds
Completion text: Step by step solution:

1. Josh bought the house for $80,000 and spent $50,000 on repairs.
2. The repairs increased the value of the house by 150%, which means the new value of the house is 100% + 150% = 250% of the original value.
3. To find the new value of the house, we can multiply the original value by 250%: $80,000 * 250% = $200,000.
4. Josh's profit is the difference between the new value of the house and the total amount he spent: $200,000 - ($80,000 + $50,000) = $200,000 - $130,000 = $70,000.

Therefore, Josh made a profit of $70,000.
